In [169]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd

vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400

In [170]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)

In [171]:
data = vbt.Data.from_data(df)
data

In [172]:
# data.plot().show()

In [173]:
open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

In [174]:
# vbt.phelp(vbt.BBANDS.run)

In [175]:
adx = vbt.ADX.run(high_price, low_price, close_price, window=14)
atr = vbt.ATR.run(high_price, low_price, close_price, window=14)
bbands = vbt.BBANDS.run(close_price, window=14)
rsi = vbt.RSI.run(close_price)
sma = vbt.MA.run(close_price, window=20)
strend = vbt.SUPERTREND.run(high_price, low_price, close_price, period=7, multiplier=3)


In [176]:
data.data['symbol']['ADX'] = adx.adx
data.data['symbol']['ATR'] = atr.atr
data.data['symbol']['BBAND'] = bbands.bandwidth
data.data['symbol']['RSI'] = rsi.rsi
data.data['symbol']['SMA'] = sma.ma
data.data['symbol']['STREND'] = strend.trend

In [177]:
data.data['symbol'] = data.data['symbol'].dropna()
predictor_list = ['Close', 'ADX', 'ATR', 'BBAND', 'RSI', 'SMA', 'STREND']
X = data.data['symbol'][predictor_list]
# print(X)

y = data.data['symbol']['signal']
y

timestamp
2022-01-02 07:00:00+00:00    SignalNone
2022-01-02 08:00:00+00:00    SignalNone
2022-01-02 09:00:00+00:00    SignalLong
2022-01-02 10:00:00+00:00    SignalNone
2022-01-02 11:00:00+00:00    SignalNone
                                ...    
2023-12-31 02:00:00+00:00    SignalNone
2023-12-31 03:00:00+00:00    SignalLong
2023-12-31 04:00:00+00:00    SignalNone
2023-12-31 05:00:00+00:00    SignalNone
2023-12-31 06:00:00+00:00    SignalNone
Freq: h, Name: signal, Length: 17472, dtype: object

In [178]:
from sklearn.model_selection import train_test_split
split_percentage = 0.7
split = int(split_percentage*len(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-split_percentage)


In [179]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,            
                                criterion='gini', 
                                max_depth=None, 
                                min_samples_split=2, 
                                min_samples_leaf=1, 
                                min_weight_fraction_leaf=0.0, 
                                max_leaf_nodes=None, 
                                min_impurity_decrease=0.0, 
                                bootstrap=True, 
                                oob_score=True, 
                                n_jobs=-1,
                                random_state=42).fit(X_train, y_train)

y_pred = clf.predict(X_test)


In [180]:
# from sklearn.metrics import classification_report
# from sklearn.metrics import accuracy_score
# report = classification_report(y_test, y_pred)
# accuracy = accuracy_score(y_test, y_pred)
# accuracy

In [181]:
df_split = data.data['symbol'][split:].copy()
df_split.loc[:, "signal"] = y_pred
signal = df_split['signal']
entries = signal == 'SignalLong'
exits = signal == 'SignalShort'

In [182]:
pf = vbt.Portfolio.from_signals(
    close=df_split.Close, 
    long_entries=entries, 
    long_exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

In [183]:
pf.plot(settings=dict(bm_returns=False)).show()

In [184]:
pf.stats()

Start                         2023-05-26 21:00:00+00:00
End                           2023-12-31 06:00:00+00:00
Period                                218 days 10:00:00
Start Value                                       100.0
Min Value                                     92.974143
Max Value                                    593.788665
End Value                                    491.085838
Total Return [%]                             391.085838
Benchmark Return [%]                         421.028013
Total Time Exposure [%]                       69.534529
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                               31.56876
Max Drawdown Duration                 101 days 21:00:00
Total Orders                                          5
Total Fees Paid                                     0.0
Total Trades                                          3
Win Rate [%]                                       50.0
Best Trade [%]                                46

In [185]:
# import joblib
# def save_model(model, model_filename='trained_model.joblib'):
#     joblib.dump(model, model_filename)
#     print(f'Model saved as {model_filename}')
# save_model(clf)